## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [36]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)

sys.path

['c:\\Users\\Public\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\python39.zip',
 'c:\\ProgramData\\Anaconda3\\DLLs',
 'c:\\ProgramData\\Anaconda3\\lib',
 'c:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\Nicolas R\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nicolas R\\.ipython',
 'C:\\Users\\Public\\random_sast\\sast',
 'C:\\Users\\Public\\random_sast\\cd_diagram',
 'C:\\Users\\Public\\random_sast\\sast',
 'C:\\Users\\Public\\random_sast\\cd_diagram']

In [37]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [38]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [39]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [40]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.892128,1.335019,0.075827,0.899491,0.074784,0.203656,Chinatown,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.950437,0.037333,0.017771,0.005390,0.010186,0.003520,Chinatown,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.979592,0.057691,0.006401,0.018758,0.027166,0.004895,Chinatown,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [41]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [42]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [43]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [44]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [45]:

filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [46]:

filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [47]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,Chinatown,0.945578,0.971817,0.958017,0.971623,0.966375,0.974927,0.982507,ROCKET
1,Coffee,0.964286,1.000000,0.978571,0.935714,1.000000,1.000000,1.000000,ROCKET
2,DistalPhalanxOutlineAgeGroup,0.734772,0.724700,0.773621,0.729017,0.729496,0.717506,0.748201,ROCKET
3,GunPoint,0.830222,0.976000,0.845778,0.972444,0.965778,0.974667,1.000000,ROCKET
4,ItalyPowerDemand,0.949660,0.946615,0.952705,0.944866,0.951668,0.944866,0.968902,ROCKET
5,SmoothSubspace,0.870222,0.910667,0.879556,0.896889,0.876000,0.908889,0.980000,ROCKET
6,SonyAIBORobotSurface2,0.751242,0.899685,0.756348,0.866387,0.837356,0.893319,0.916055,ROCKET
7,SyntheticControl,0.920000,0.967333,0.896000,0.963333,0.981333,0.966667,1.000000,ROCKET


In [48]:
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
len_method_vs_ds

len_method,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF
0,Chinatown,0.000856,0.000100,0.000323,0.000162,0.000167,0.000062
1,Coffee,0.000000,0.000000,0.000383,0.000255,0.000000,0.000000
2,DistalPhalanxOutlineAgeGroup,0.000361,0.000144,0.000324,0.000175,0.000316,0.000203
3,GunPoint,0.000526,0.000088,0.000234,0.000050,0.000145,0.000097
4,ItalyPowerDemand,0.000109,0.000015,0.000055,0.000051,0.000029,0.000038
5,SmoothSubspace,0.001110,0.000316,0.000187,0.000266,0.001008,0.000341
6,SonyAIBORobotSurface2,0.004930,0.000157,0.003355,0.000337,0.001183,0.000283
7,SyntheticControl,0.000572,0.000013,0.000136,0.000039,0.000048,0.000017


In [49]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.853704,0.778435,0.837692,0.859550,0.853497,0.852777
ACF&PACF,0.909863,0.880221,0.905329,0.910962,0.909766,0.909838
Max ACF,0.864338,0.791108,0.836910,0.865115,0.853144,0.862739
Max PACF,0.899804,0.852826,0.901013,0.897054,0.899552,0.898599
None,0.892311,0.851442,0.890546,0.891022,0.887222,0.893171
PACF,0.908166,0.883419,0.904938,0.908277,0.909211,0.909048


In [50]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.852277,0.747302,0.839994,0.869009,0.861734,0.866291
ACF&PACF,0.912432,0.860737,0.902135,0.916448,0.916225,0.916289
Max ACF,0.860661,0.751721,0.847705,0.873397,0.869955,0.872364
Max PACF,0.900721,0.839823,0.896846,0.900232,0.906379,0.904235
None,0.895266,0.813522,0.889489,0.900719,0.903773,0.900976
PACF,0.910053,0.869073,0.903005,0.913345,0.911645,0.914681


#### Time: Subsequence Lenght Method 

In [51]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,GunPoint,ItalyPowerDemand,SmoothSubspace,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,
ACF,0.073226,41.772693,151.661253,37.041264,0.353901,0.561044,0.700003,81.701994
ACF&PACF,0.156653,134.427490,411.991204,71.945396,0.729001,0.930945,2.032898,198.457047
Max ACF,0.076082,3.028503,54.085258,4.165960,0.361998,0.565765,0.548111,28.897669
Max PACF,0.069733,6.288893,27.972717,4.032023,0.387409,0.639174,0.413045,16.967036
None,0.054860,1.243746,25.488523,2.376824,0.331860,0.523768,0.305284,15.313601
PACF,0.106152,103.524388,290.194187,47.107583,0.711325,0.843548,1.655537,153.311054


In [52]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,49.248066,2.065920,12.551974,60.105313,32.202674,42.542930
ACF&PACF,132.092443,3.743905,31.707417,155.749101,79.898248,107.494867
Max ACF,15.406853,0.848845,3.528608,18.971306,8.873957,12.544427
Max PACF,9.135196,0.833218,2.442551,10.258583,5.365170,7.141228
None,7.584766,0.787730,1.826960,8.982832,4.388182,5.973253
PACF,94.975194,2.691102,22.203440,110.806881,56.174852,76.061881


In [53]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,46.127762,1.584537,9.348424,70.673232,27.284497,45.778627
ACF&PACF,129.337362,4.134285,26.186833,156.501340,76.111523,120.251361
Max ACF,13.344738,0.868442,3.358362,20.920895,8.741030,14.315272
Max PACF,9.049460,1.350768,2.840182,8.164095,6.010496,7.818101
None,6.924661,0.760475,1.989226,8.489867,4.722618,7.096947
PACF,95.698235,2.908977,19.264268,102.921231,55.705996,85.932618


#### Variance: Subsequence Lenght Method 

In [54]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009432,0.019491,0.010639,0.009099,0.009327,0.008700
ACF&PACF,0.006958,0.009771,0.008140,0.007527,0.007620,0.007879
Max ACF,0.008716,0.016977,0.010500,0.008202,0.008564,0.008511
Max PACF,0.006815,0.014530,0.006539,0.008587,0.006847,0.007682
None,0.007811,0.014358,0.007852,0.007990,0.007913,0.008019
PACF,0.007672,0.009808,0.008421,0.009035,0.008617,0.008622


In [55]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.010004,0.019554,0.012038,0.009101,0.011073,0.010598
ACF&PACF,0.007721,0.009743,0.007878,0.007851,0.008121,0.007791
Max ACF,0.008508,0.018962,0.012280,0.009437,0.009861,0.009535
Max PACF,0.007286,0.013385,0.006883,0.008512,0.007396,0.008688
None,0.008374,0.014397,0.008438,0.009369,0.008938,0.008894
PACF,0.008297,0.010525,0.008369,0.008795,0.008238,0.008558


#### Another Results

In [56]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,4800,4800,4800
unique,8,6,216
top,Chinatown,ACF&PACF,Max ACF: n_random_points= (lenthg ts)//2 nb_in...
freq,720,800,100


In [57]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Chinatown,0.017890
Coffee,3.225397
DistalPhalanxOutlineAgeGroup,32.046438
GunPoint,5.555635
ItalyPowerDemand,0.095850
SmoothSubspace,0.135475
SonyAIBORobotSurface2,0.188496
SyntheticControl,5.496093


In [58]:
df_result['time'].sum()/(60*60)

46.76127400689655

In [139]:
ints='\(max instances per class\)//2'
len_m='ACF&PACF'
df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':')]
df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r'nb_inst_per_class='+ints+'$')]
df_to_cd.classifier_name.unique()

array(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=(max instances per class)//2',
       'ACF&PACF: n_random_points=1 nb_inst_per_class=(max instances per class)//2',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=(max instances per class)//2',
       'ACF&PACF: n_random_points=100 nb_inst_per_class=(max instances per class)//2',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=(max instances per class)//2',
       'ACF&PACF: n_random_points=50 nb_inst_per_class=(max instances per class)//2'],
      dtype=object)

#### Generate CD Diagram Tunning Hyperparameter

In [150]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
for k,ints in enumerate(df_result.nb_per_class.unique()):
   ints_r=ints.replace("(","\(")
   ints_r=ints_r.replace(")","\)")
   for len_m in df_result.len_method.unique():
      df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints_r)+'$')]
      print('len_method'+str(len_m))
      print('nb_per_class'+str(ints))
      draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison, nb_inst_per_class="+str(ints), name_file='images_cd_diagram/cd-diagram_'+len_m+str(k)+'.png')
      #draw_cd_diagram(, labels=True, title="ACF&PACF comparison nb_inst_per_class="+str(j), name_file='images_cd_diagram/cd-diagram_ACF&PACFnb_inst_per_class='+str(j)+'.png')
      #draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
      #draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
      #draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
      #draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
      #draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
      #draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



len_methodACF&PACF
nb_per_class(max instances per class)//2


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodACF
nb_per_class(max instances per class)//2


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodPACF
nb_per_class(max instances per class)//2
the null hypothesis over the entire classifiers cannot be rejected


...

c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodMax PACF
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodNone
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [60]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,214.7500
ACF: n_random_points=1 nb_inst_per_class=1,213.2500
Max ACF: n_random_points=10 nb_inst_per_class=1,207.3125
ACF: n_random_points=10 nb_inst_per_class=1,204.5000
Max PACF: n_random_points=1 nb_inst_per_class=1,204.1250
...,...
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,54.1250
ACF&PACF: n_random_points=100 nb_inst_per_class=(max instances per class)//2,54.0000
ACF&PACF: n_random_points=30 nb_inst_per_class=100,53.0000
ACF&PACF: n_random_points=100 nb_inst_per_class=50,50.4375


In [61]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [62]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50'], dtype='object')

In [77]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,0.912789,0.869393,0.901338,0.920097,0.923133,0.920720
1,0.899449,0.791263,0.885582,0.908069,0.895154,0.901808
10,0.909259,0.861650,0.908505,0.916423,0.917684,0.918451
30,0.918103,0.879780,0.904196,0.914956,0.920776,0.920786
50,0.913824,0.880732,0.908333,0.919345,0.918831,0.917962
100,0.919740,0.881601,0.904856,0.918460,0.922153,0.918959


In [64]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.007344,0.007349,0.008620,0.007569,0.008514,0.008277
1,0.005389,0.018353,0.006680,0.006496,0.005984,0.007831
10,0.008020,0.007107,0.007133,0.007681,0.008525,0.008367
100,0.006907,0.007060,0.009669,0.008040,0.008131,0.008082
30,0.006754,0.008922,0.009429,0.007986,0.008695,0.007744
50,0.007255,0.005984,0.009068,0.009095,0.007772,0.008464


##### Overall time complexity

In [65]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
Chinatown,0.000148,0.001009,0.000235,0.001189,0.002611
Coffee,0.001126,0.082135,0.000738,2.156343,2.240458
DistalPhalanxOutlineAgeGroup,0.000529,0.020612,0.015880,6.829225,6.866520
GunPoint,0.000645,0.027494,0.000965,1.169863,1.199090
ItalyPowerDemand,0.000136,0.001336,0.000842,0.009811,0.012150
SmoothSubspace,0.000103,0.002251,0.002864,0.010263,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.000283,0.030471,0.033882
SyntheticControl,0.000596,0.022403,0.007352,3.277018,3.307617


In [66]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,2.594967,0.081025,0.793013,1.867173,2.403174,3.437006
1,0.103583,0.015111,0.056451,0.061058,0.075114,0.102111
10,0.618486,0.049502,0.154839,0.407523,0.564583,0.823751
30,1.813811,0.048544,0.450354,1.193613,1.661248,2.443583
50,2.888689,0.079079,0.704976,1.892084,2.611614,3.848694
100,3.616004,0.101130,1.011108,2.568373,3.433753,4.919766


##### Calculate weights time complexity

In [67]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.022103,0.025822,0.027434,0.020311,0.025918,0.022845
1,0.030458,0.020644,0.020792,0.022511,0.023626,0.023475
10,0.022519,0.021759,0.020989,0.019447,0.017650,0.022379
100,0.021180,0.020841,0.017924,0.022874,0.019713,0.023209
30,0.025112,0.023778,0.020492,0.022524,0.018971,0.022536
50,0.026394,0.018172,0.019975,0.022934,0.015305,0.023215


##### Finding subsequences time complexity

In [68]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.965750,0.092316,0.733471,1.434441,1.292325,1.442339
1,1.569111,0.162715,0.497933,1.041109,0.837010,0.914812
10,0.887332,0.043997,0.405674,0.992672,0.693998,0.847386
100,0.898612,0.052281,0.464582,1.253262,0.958145,1.196144
30,0.876790,0.050819,0.461581,1.114042,0.823055,1.007264
50,0.986909,0.044196,0.485267,1.207313,0.889239,1.027739


##### Transform Dataset time complexity

In [69]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,154.544750,4.573416,46.756754,204.545282,110.572247,142.527730
1,4.360863,0.346049,1.110141,5.035816,2.632361,3.537461
10,36.138123,1.065136,8.833083,48.350240,23.698831,32.949706
100,215.845850,5.875026,60.121621,293.667823,152.983376,204.605143
30,107.781804,2.705348,26.493982,145.328837,70.693309,98.532441
50,172.122037,4.543585,41.738543,229.488478,112.511599,155.503506


##### Classifier time complexity

In [70]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.152924,0.168606,0.058404,0.207417,0.130484,0.184439
1,0.249031,0.375805,1.755404,0.025696,0.167911,0.029237
10,0.058772,1.837423,0.028465,0.059958,0.038723,0.053221
100,0.186279,0.117877,0.059439,0.230976,0.134306,0.192364
30,0.139268,0.130812,0.041478,0.143987,0.077672,0.107968
50,0.178300,0.136425,0.051516,0.194202,0.103430,0.135921
